<a href="https://colab.research.google.com/github/fouad89/arabicNLP/blob/main/fine_tuning_Arabert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 9.6 MB/s 
     |████████████████████████████████| 6.6 MB 42.8 MB/s 
     |████████████████████████████████| 596 kB 50.7 MB/s 
     |████████████████████████████████| 86 kB 6.2 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [ ]:
!pip install farasapy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!git clone https://github.com/aub-mind/arabert.git

Cloning into 'arabert'...
remote: Enumerating objects: 569, done.
remote: Counting objects: 100% (34/34), done.
remote: Compressing objects: 100% (11/11), done.
remote: Total 569 (delta 26), reused 24 (delta 23), pack-reused 535
Receiving objects: 100% (569/569), 9.12 MiB | 23.71 MiB/s, done.
Resolving deltas: 100% (327/327), done.


In [ ]:
!pip install pyarabic

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 126 kB 8.2 MB/s 


## sentiment using pipeline

In [ ]:
# using pipeline
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import torch
import torch.nn.functional as F

In [ ]:
# arabert model from hugging face https://huggingface.co/aubmindlab/bert-large-arabertv2
model_name = "aubmindlab/bert-large-arabertv2"
model = AutoModelForSequenceClassification.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Downloading:   0%|          | 0.00/385 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.38G [00:00<?, ?B/s]

Some weights of the model checkpoint at aubmindlab/bert-large-arabertv2 were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification wer

Downloading:   0%|          | 0.00/611 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/641k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.20M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
classifier = pipeline('sentiment-analysis', model=model, tokenizer=tokenizer)

In [ ]:
sample_text = ["أهنئ الدكتور أحمد جمال الدين، القيادي بحزب مصر، بمناسبة صدور أولى روايته", # pos
               "انا لو نفسي اخطط و ارتب و اصرف فلوس الدنيا عشان اشوه الاخوان مش حاعرف اخترع حد زي صفوت حجازي", # neg
               "امير عيد هو اللي فعلا يتقال عليه ستريكر صريح #كاريوكي #السكة_شمال", # POS,
               "_ 0 مش عيب تكذب؟ عمري ما عملت ولا هعمل بلوك لحد لا تمدح نفسك", # 	NEG
]

In [ ]:
results = classifier(sample_text)

In [ ]:
# print the results
for text, s in zip(sample_text, results):
    print(f"{text} classified as: {s}")

أهنئ الدكتور أحمد جمال الدين، القيادي بحزب مصر، بمناسبة صدور أولى روايته classified as: {'label': 'LABEL_0', 'score': 0.6027821898460388}
انا لو نفسي اخطط و ارتب و اصرف فلوس الدنيا عشان اشوه الاخوان مش حاعرف اخترع حد زي صفوت حجازي classified as: {'label': 'LABEL_1', 'score': 0.5439120531082153}
امير عيد هو اللي فعلا يتقال عليه ستريكر صريح #كاريوكي #السكة_شمال classified as: {'label': 'LABEL_0', 'score': 0.6448686122894287}
_ 0 مش عيب تكذب؟ عمري ما عملت ولا هعمل بلوك لحد لا تمدح نفسك classified as: {'label': 'LABEL_0', 'score': 0.5682076215744019}


In [ ]:
text2 = "انا احب اللغة"

In [ ]:
classifier(text2)

[{'label': 'LABEL_1', 'score': 0.5453447103500366}]

In [ ]:
## Tokens

In [ ]:
batch = tokenizer(sample_text, padding=True, truncation=True, max_length=512, return_tensors="pt")

In [ ]:
with torch.no_grad():
    outputs = model(**batch)
    print(outputs)
    predictions = F.softmax(outputs.logits, dim=1)
    print(predictions)
    labels = torch.argmax(predictions, dim=1)
    print(labels)
    # convert labels 
    labels = [model.config.id2label[label_id] for label_id in labels.tolist() ]
    print(labels)


SequenceClassifierOutput(loss=None, logits=tensor([[ 0.6462,  0.2291],
        [ 0.1190,  0.2951],
        [ 0.5063, -0.0903],
        [ 0.4617,  0.1871]]), hidden_states=None, attentions=None)
tensor([[0.6028, 0.3972],
        [0.4561, 0.5439],
        [0.6449, 0.3551],
        [0.5682, 0.4318]])
tensor([0, 1, 0, 0])
['LABEL_0', 'LABEL_1', 'LABEL_0', 'LABEL_0']


## Using Arabert Preprocessor

In [ ]:
from arabert.preprocess import ArabertPreprocessor


In [ ]:
arabert_prep = ArabertPreprocessor(model_name=model_name)

/usr/local/lib/python3.7/dist-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


100%|██████████| 241M/241M [00:22<00:00, 10.6MiB/s]


[2022-06-02 09:23:52,898 - farasapy_logger - WARNING]: Be careful with large lines as they may break on interactive mode. You may switch to Standalone mode for such cases.


In [ ]:
prepped_text = arabert_prep.preprocess(sample_text)

In [ ]:
results2 = classifier(prepped_text)

In [ ]:
for r in results:
    print(r)

{'label': 'LABEL_0', 'score': 0.6027821898460388}
{'label': 'LABEL_1', 'score': 0.5439120531082153}
{'label': 'LABEL_0', 'score': 0.6448686122894287}
{'label': 'LABEL_0', 'score': 0.5682076215744019}
